## pair coding environment

As graduates of Wyncode Academy, we want to be able to pair code while not sitting next to each other on the same network.

At least one person does not edit text files in the terminal.

Specifications:

* ability to use voice chat

**Trivially solved with any app that supports voice chatting.**

* ability to see results of executed code: 

**Easily solved with shared tmux session after users are granted shell access via ssh.**

* ability for all users to have immediately available sourced files that they can edit with any text editor of choice: 

*Not as trivial as the first 2 specs.*

## possible solutions

### combination of the following:

* git: https://git-scm.com/
* gitolite: http://gitolite.com/gitolite/index.html
  
  gitolite is for hosting git repositories.
  
* Python package watchdog: https://pypi.python.org/pypi/watchdog

  For watching a directory and initiating an automated series of steps after file changes. 
    
  The file changes trigger a commit and push to the git repo from which all other users can pull. 
    
  The pull can be initiated by using watchdog to watch the designated gitolite repo

#### setup

* Install a virtualenv with python [`watchdog`](https://pypi.python.org/pypi/watchdog) package installed.
* Set up a named repo for a project in the gitolite hosting git repository. This must be done by a gitolite administrator by editing `~/gitolite-admin/conf/gitolite.conf`. 

In this example let's call the new pair coding repo `$new_project_name`.

In [5]:
new_project_name = 'trial-setup'

It is possible to view the current repos with this command:

In [2]:
!./bin/gitolite-info

hello dmmmd, this is gitolite@zip running gitolite3 v3.6.6-2-g8620d5f on git 1.9.1

 R W	gitolite-admin
 R W	pair-coding-template
 R W	practice
 R W	test-project
 R W	testing


* clone the template repo that will become the working project repo set up earlier

In [11]:
!./bin/git-clone $new_project_name

Cloning into 'trial-setup'...
remote: Counting objects: 207, done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 207 (delta 98), reused 190 (delta 94)
Receiving objects: 100% (207/207), 26.64 KiB | 0 bytes/s, done.
Resolving deltas: 100% (98/98), done.
Checking connectivity... done.


The cloned directory contains scripts to set up the direcotry for pair coding.

`cd` into the directory

In [7]:
!cd $new_project_name

* Run the setup executable: 

```bash
./setup "$new_project_name"
```



Then run:

```bash
./watchdog
```

And the watching should begin. All file changes should result in a commit and push.

### More about the files

#### watchdog file

In [22]:
# %load watchdog

```bash
#!/bin/bash

./automated_git.sh 2>&1 > watchdog.log &  #  start watchdog and redirect output to log and put in background
```

In [23]:
# %load automated_git.sh

```bash
#!/bin/bash
watchmedo shell-command \
    --patterns='*' \
    --ignore-patterns='.git *.log' \  # avoid race condition when these files update from the update
    --command='git add -A && git commit -m replaced_by_hook && git push origin master'
```

* The process with watchdog has to be manually killed. This is done with the following script:

```bash
./get-watchdog-pid
```

* Then kill the process:

```bash
kill $pid
```

* This will kill all the watchdog processes.

```bash
./kill-all-watchdogs
```

#### expected result

Every time a file other than those in the ignore-patterns is changed it will trigger a commit and push to the gitolite repo where it can be pulled by other users.

#### todo

* Set up vim key mapping so when a non-terminal edit has occurred the repo is pulled and vim files are reloaded.

## Extras

* rst directory: contains an rst file to use with https://github.com/Rykka/InstantRst

As that file is updated another user can copy and paste items from it. Sometimes it's easier than saying what should be typed. Can be used as a scratch pad, too. It is instantly updated to all those subscribed to the web page it produces.